In [45]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [164]:
def balloon_coin_popping(a: list[int]) -> int:
    a = [1, *a, 1]

    @lru_cache(None)
    def sub(i: int, j: int) -> int:
        m = lambda k: sub(i, k) + a[i - 1] * a[k] * a[j] + sub(k + 1, j)
        return max(map(m, range(i, j))) if i < j else 0

    return sub(1, len(a) - 1)

In [179]:
def balloon_coin_popping(a: list) -> int:
    a = [1, *a, 1]

    @lru_cache(None)
    def sub(i: int, j: int) -> int:
        return i < j - 1 and max(
            sub(i, k) + a[i] * a[k] * a[j] + sub(k, j) for k in range(i + 1, j)
        )

    return sub(0, len(a) - 1)

In [180]:
balloon_coin_popping([5, 10])

60

In [181]:
balloon_coin_popping([3, 1, 5, 8])

167